IMPORTING MODULES

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms

import requests
import zipfile
import pathlib
from pathlib import Path

import random 
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

import os

from typing import Tuple, Dict, List

HYPERPARAMETERS

In [2]:
INPUT_SHAPE = 0
HIDDEN_UNITS = 0
OUPUT_SHAPE = 3

BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count()
EPOCHS = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Downloading Dataset

In [ ]:
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory already exists")
else:
    print(f"creating {image_path}...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print(f"Unziping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

print("DONE!")


SETTING DATA

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

random.seed(42)
# Get all image paths
image_path_list = list(image_path.glob("*/*/*.jpg"))

# Pick a random image path
random_image_path = random.choice(image_path_list)

# Get image class from path name 
image_class = random_image_path.parent.stem
print(image_class)

# Open image
img = Image.open(random_image_path)

# Print metadata
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")

img_as_array = np.array(img)

plt.figure(figsize=(5, 5))
plt.imshow(img_as_array)
plt.title(f"Image class: {image_class} | Image_shape: {img_as_array.shape}")
plt.axis(False)

CREATING A CUSTOM DATASET

In [5]:
class ImageFolderCustom(Dataset):
    
    def __find_classes(self, directory: str) -> Tuple[List[str], Dict[str, int]]:
        # Get the class names by scanning the target directory
        classes = sorted([entry.name for entry in list(os.scandir(directory))])
        
        # Raise an error if class names coult not be found
        if not classes:
            raise FileNotFoundError(f"!Couldn't find any classes in {directory}!\nPlease check file structure")
        
        # Create a dictionary of index labels
        class_to_idx = {class_name: i for i, class_name in enumerate(classes)}
        
        return classes, class_to_idx

    def __init__(self, targ_dir: str, transform=None):
        # Get all of the image paths
        self.paths = list(pathlib.Path(targ_dir).glob("*/*.jpg"))
        # Setup transform
        self.transform = transform
        # Create classes and class_to_idx attributes
        self.classes, self.class_to_idx = self.__find_classes(targ_dir)
    
    def load_image(self, index: int) -> Image.Image:
        image_path = self.paths[index]
        return Image.open(image_path)
    
    def __len__(self) -> int:
        return len(self.paths)
    
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        img = self.load_image(index)
        class_name = self.paths[index].parent.name
        class_idx = self.class_to_idx[class_name]
        
        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, class_idx


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

train_data_custom = ImageFolderCustom(targ_dir=train_dir,
                                      transform=train_transform)
test_data_custom = ImageFolderCustom(targ_dir=test_dir,
                                     transform=test_transform)

train_dataloader = DataLoader(dataset=train_data_custom,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data_custom,
                             batch_size=8,
                             num_workers=0,
                             shuffle=False)

img, label = next(iter(train_dataloader))
img, label

VISUALIZING DATASET

In [9]:
def display_random_images(dataset: torch.utils.data.Dataset,
                          classes: List[str] = None,
                          n: int = 10,
                          display_shape: bool = True,
                          seed: int = None):
    # Adjust display if n is too high
    if n > 10:
        n = 10
        display_shape = False
        print(f"For display purposes, n shouldn't be larger than 10. Setting to 10 and removing shape display.")

    # Set the seed
    if seed:
        random.seed(seed)

    # Get random sample indexes
    random_samples_idx = random.sample(range(len(dataset)), k=n)

    # Setup plot
    plt.figure(figsize=(16, 8))

    # Loop through random indexes and plot them
    for i, targ_sample in enumerate(random_samples_idx):
        targ_image, targ_label = dataset[targ_sample][0], dataset[targ_sample][1]

        # Adjust tensor dimensions for plotting
        targ_image_adjust = targ_image.permute(1, 2, 0)

        # Plot adjusted samples
        plt.subplot(1, n, i+1)
        plt.imshow(targ_image_adjust)
        plt.axis("off")
        if classes:
            title = f"Class: {classes[targ_label]}({targ_sample})"
            if display_shape:
                title = title + f"\nShape: {targ_image_adjust.shape}"
        plt.title(title)  

In [ ]:
display_random_images(train_data_custom,
                      n=5,
                      classes=train_data_custom.classes,
                      seed=None)